## Users 

### Upgrade date rank
- From Silver Users
- Rank the number of times a user switches from a nonpaying status to a paying status

### Upgrade Never
- From Silver User
- Users who have never paid on the platform

### Email Unsubs
- From Braze table
- A unsubscribe event before the report date (end of the month)

### Email Targeted
- From Braze table
- Received an email in this month

### Holdout (AFTER JULY 2021)
- From holdout table
- In current holdout cohort
- Excluses users who received an email in this month

### Email Engagers
- From Braze Marketing View
- Has an Adobe ID
- Non-Machine non-transactional email open before the report date (end of the month)

### All Cohorts
- From Braze Marketing View
- In Email Targeted or Holdout, and Silver Users
- Exclude Abandoned MAAs
- Exclude Unsubscribes

### Video Viewing
- From All Cohorts
- In Silver Video in the current month

In [ ]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_KPI_Results_Feb_23` AS

WITH 
upgrade_date_rank AS (
    SELECT adobe_tracking_id
            , report_date
            , row_number() OVER(partition by adobe_tracking_id order by report_date ) as upgrade_row_number -- rank the number of times a user upgrade
    FROM       
            (
                SELECT adobe_tracking_id
                        , report_date
                        , paying_account_flag as paying_account_flag_today
                        , LAG(paying_account_flag,1) OVER ( partition by adobe_tracking_id order by report_date  ) as paying_account_flag_yestd -- paying flag yesterday
                FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
                --WHERE report_date between '2022-01-01' and '2022-07-10' -- don't add report_date since we need to distinguish upgrades from beginnning of time 
                ORDER BY 1,2    -- testing
            )
    WHERE paying_account_flag_today = 'Paying' AND paying_account_flag_yestd = 'NonPaying'
)
,upgrade_never AS (
    SELECT adobe_tracking_id
        , report_date
    FROM (
            SELECT adobe_tracking_id
                    , report_date
                    , sum(case when paying_account_flag = 'Paying' then 1 else 0 end) OVER(partition by adobe_tracking_id order by report_date ) as cumulative_nonpaying_num -- rank the number of times a user upgrade
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
        )
    WHERE cumulative_nonpaying_num = 0
)
,Email_Unsubs AS (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Unsubscribes' 
    AND event_date <= @report_end_date
)
,Email_Targeted AS ( -- everyone who have received emails in the month, and opened at least one email in the past
    SELECT DISTINCT adobe_tracking_id 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Deliveries' 
    AND event_date BETWEEN @report_start_date and @report_end_date
)
--HOLDOUT 2.0, for months after July 2021
,Holdout AS ( 
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as first day of quarter
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        -- Exclude those who are assigned to Email Holdout but actually received emails 
    ) holdout
    LEFT JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries'
        AND event_date >= @report_start_date
        AND event_date <= @report_end_date 
    ) delivered -- Email Delivered ever
    ON holdout.adobe_tracking_id = delivered.adobe_tracking_id AND delivered.adobe_tracking_id is null
)
,All_Cohorts AS (
    -- email targetable and holdout cohort, users signed up after 2020/8/11
     SELECT distinct a.adobe_tracking_id as aid
     , cohort
     , user.account_type

    FROM         -- Include email engagers only: users who have at least 1 'Other Opens' in entire user histoy; exclude Oct 2022 Privacy email
     (SELECT DISTINCT identity 
            FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
            WHERE eventName = 'Email Opens' and machineOpen is null
            AND date(eventTimestamp) <= @report_end_date
            AND campaignName NOT IN ('TransactionalTermsOfUseEngaged20221028', 'TransactionalTermsOfUseNONEngaged20221028') -- Exclude email openers of Oct privacy email
        ) Email_Engagers --opened email at least once, using other open 

          INNER JOIN `nbcu-ds-sandbox-a-001.sl_sandbox.Braze_Id_Adobe_Id_Map` mapping
          ON mapping.bid = Email_Engagers.identity

          INNER JOIN
              (SELECT *, 'Email_Targeted' as cohort from Email_Targeted
                  UNION ALL
                  SELECT *, 'Holdout' as cohort from Holdout
              ) a
          ON a.adobe_tracking_id = mapping.aid
              
          --add attribute: account_type 
          INNER JOIN ( SELECT * FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` WHERE report_date = @report_end_date ) user
          ON a.adobe_tracking_id = user.adobe_tracking_id 
        

        -- for after 2021/july, email channel only, take out all abandon MAAs
        INNER JOIN 
        (SELECT DISTINCT adobe_tracking_id FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
            WHERE report_date = @report_end_date
            AND date_of_last_view IS NOT NULL 
        ) abandon_maa 
        ON a.adobe_tracking_id = abandon_maa.adobe_tracking_id

        -- exclude unsubscribed
        LEFT JOIN Email_Unsubs unsub
        ON a.adobe_tracking_id = unsub.adobe_tracking_id
        WHERE unsub.adobe_tracking_id is null
)
,Video_Viewing AS (
    SELECT 
        adobe_tracking_id
        , COUNT (DISTINCT CASE WHEN VIDEO.num_views_started =1 THEN video.adobe_tracking_id ELSE NULL END) AS Distinct_Content_Starts
        , SUM (VIDEO.num_views_started ) AS Total_Content_Starts
        , SUM(VIDEO.num_seconds_played_no_ads)/3600 AS Viewing_Time
        , COUNT(DISTINCT session_id) AS Distinct_Viewing_Sessions 
        , COUNT(DISTINCT(CASE WHEN (num_seconds_played_no_ads > CASE WHEN lower(consumption_type) = 'virtual channel' THEN 299 ELSE 0 END)
                and (num_views_started>0) THEN CASE WHEN (lower(consumption_type) = "shortform") THEN "Shortform"
                                                    WHEN lower(franchise) != 'other' THEN franchise ELSE display_name
                                                    END
                end)) as Repertoire_Pavo_Method
                
    FROM All_Cohorts a

    INNER JOIN  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO
        ON VIDEO.adobe_tracking_id = a.aid
        AND adobe_date between @report_start_date AND @report_end_date
    GROUP BY 1
)

Attributes/User Features:

Most features derived from SILVER_USER at the end of the month

- primary_device: From SILVER_PRIMARY_DEVICE
- account_tenure
- account_type
- active_viewer: viewership status (or null) in engagement 9box table
- billing_cycle_category
- billing_platform
- bundling_partner
- churn_frequency: Number of times they have churned, in 3 buckets
- offer: on an offer
- paid_tenure: tenure_paid_lens if paying user
- paying_account_flag: paying status from SILVER_USER
- intender_audience franchise: ia franchise, keep name as IA
- genre: ia genre
- network: ia network
- previously_bundled: bundling_partner is null in history

# KPIs

- `audience` table refers to group of users 
- `viewer` table defines viewership metrics of all users, defined in **Viewer Metrics**
- All divisions are `safe_divide`, but are excluded from logic for clarity
- All measures assume distinct adobe_tracking_id, but `DISTINCT` keyword is added for clarity

### Viewer Metrics
- Viewing table defined monthly
- Excludes ads, trailers, and autoplay (same logic as video_watched_trailing_30d in SILVER_USER) 

```
SELECT  adobe_tracking_id
    ,COUNT(DISTINCT CASE WHEN num_views_started = 1 THEN adobe_tracking_id ELSE NULL END) AS Distinct_Content_Starts -- num_views_started is a flag
    ,SUM(num_views_started)                                                               AS Total_Content_Starts
    ,SUM(num_seconds_played_no_ads)/3600                                                  AS Viewing_Time
    ,COUNT(DISTINCT CASE WHEN num_views_started = 1 THEN session_id ELSE NULL END)        AS Distinct_Viewing_Sessions
    ,COUNT(DISTINCT(CASE 
                        WHEN (num_seconds_played_no_ads > CASE 
                                                            WHEN LOWER(consumption_type) = 'virtual channel' THEN 299 
                                                            ELSE 0 
                                                            END
                                ) AND (num_views_started > 0) 
                        THEN CASE 
                                WHEN (LOWER(consumption_type) = "shortform") THEN "Shortform"
                                WHEN LOWER(franchise) != 'other' THEN franchise 
                                ELSE display_name
                                END
                    END)
        )                                                                                    AS Repertoire_Pavo_Method
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`
WHERE (adobe_date BETWEEN @report_start_date AND @report_end_date)
AND (num_views_started = 1)
AND NOT(
    (COALESCE(stream_type,"NULL") = 'trailer') 
    AND (
        COALESCE(LOWER(vdo_initiate),"NULL") LIKE ('%auto%play%')
        --From 2023-03-01 onwards vdo_initiate=n/a will be considered as Auto play
        OR 
        (adobe_date >= '2023-03-01' AND COALESCE(LOWER(vdo_initiate),"NULL") = 'n/a')
    )
)
GROUP BY 1
```

### Return Rate
- Percentage of the cohort with a video watch activity sometime in the reporting period.
- Num: Users with video watch activity (Silver Video) 
- Denom: Cohort Size
- Dependencies: `Video_Viewing`

```
SELECT COUNT(DISTINCT v.adobe_tracking_id) / COUNT(DISTINCT a.adobe_tracking_id)
FROM audience a
LEFT JOIN viewing v
ON a.adobe_tracking_id = v.adobe_tracking_id
```

### Usage
- Average viewing time of all users in the cohort
- Num: Sum of all user viewing time
- Denom: Cohort Size
- Dependencies: `Video_Viewing`

```
SELECT SUM(v.Viewing_Time) / COUNT(DISTINCT a.adobe_tracking_id)
FROM audience a
LEFT JOIN viewing v
ON a.adobe_tracking_id = v.adobe_tracking_id
```

### Repertoire
- Average repertoire of all users in the cohort
- Num: Sum of all user repertoire
- Denom: Cohort Size
- Dependencies: `Video_Viewing`

```
SELECT SUM(v.Repertoire_Pavo_Method) / COUNT(DISTINCT a.adobe_tracking_id)
FROM audience a
LEFT JOIN viewing v
ON a.adobe_tracking_id = v.adobe_tracking_id
```

### Viewing Sessions
- Average number of distinct viewing sessions of all users in the cohort
- Num: Sum of all distinct user viewing sessions
- Denom: Cohort Size
- Dependencies: `Video_Viewing`

```
SELECT SUM(v.Distinct_Viewing_Sessions) / COUNT(DISTINCT a.adobe_tracking_id)
FROM audience a
LEFT JOIN viewing v
ON a.adobe_tracking_id = v.adobe_tracking_id
```

### Lapsed Save Rate
- Percentage of the cohort who were past lapsing (last watch date between 30 and 90 days) sometime during the reporting period that had watched something in the reporting period
- Num: Users who watched something in the current month who were past lapsing (last watch date is between 30 and 90 days)
- Denom: Users in the month who are past lapsing (last watch date is between 30 and 90 days)
- Dependencies: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`, `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`

```
WITH 
    Lapsed_Save_Base AS (
        SELECT  DISTINCT adobe_tracking_id
            ,date_of_last_view
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
        WHERE (report_date BETWEEN report_start_date AND report_end_date)
        AND (days_since_last_view BETWEEN 30 AND 90)
    )
    , Lapsed_Save_Denom AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM Lapsed_Save_Base
    )
    , Lapsed_Save_Num AS (
        SELECT  DISTINCT a.adobe_tracking_id
        FROM Lapsed_Save_Base a
        INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` video
        ON video.adobe_tracking_id = a.adobe_tracking_id
        AND (adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 30 day) AND DATE_ADD(date_of_last_view, INTERVAL 90 day))
        AND (adobe_date BETWEEN report_start_date AND report_end_date)
        AND (num_views_started = 1)
        AND NOT(
            (COALESCE(stream_type,"NULL") = 'trailer') 
            AND (
                COALESCE(LOWER(vdo_initiate),"NULL") LIKE ('%auto%play%')
                --From 2023-03-01 onwards vdo_initiate=n/a will be considered as Auto play
                OR 
                (adobe_date >= '2023-03-01' AND COALESCE(LOWER(vdo_initiate),"NULL") = 'n/a')
            )
        )
    )
SELECT COUNT(DISTINCT Lapsed_Save_Num.adobe_tracking_id) / COUNT(DISTINCT Lapsed_Save_Denom.adobe_tracking_id)
FROM audience a 
LEFT JOIN Lapsed_Save_Denom
    ON a.adobe_tracking_id = Lapsed_Save_Denom.adobe_tracking_id
LEFT JOIN Lapsed_Save_Num
    ON Lapsed_Save_Denom.adobe_tracking_id = Lapsed_Save_Num.adobe_tracking_id
```

### Lapsing Save Rate
- Percentage of the cohort who were past lapsing (last watch date between 15 and 29 days) sometime during the reporting period that had watched something in the reporting period
- Num: Users who watched something in the current month who were lapsing users (last watch date is betweeen 15 and 29 days)
- Denom: Users in the month who are lapsing users (last watch date is betweeen 15 and 29 days)
- Dependencies: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`, `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`

```
WITH 
    Lapsing_Save_Base AS (
        SELECT  DISTINCT adobe_tracking_id
            ,date_of_last_view
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
        WHERE (report_date BETWEEN report_start_date AND report_end_date)
        AND (days_since_last_view BETWEEN 15 AND 29)
    )
    , Lapsing_Save_Denom AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM Lapsing_Save_Base
    )
    , Lapsing_Save_Num AS (
        SELECT  DISTINCT a.adobe_tracking_id
        FROM Lapsing_Save_Base a
        INNER JOIN  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` video
        ON video.adobe_tracking_id = a.adobe_tracking_id
        AND (adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 15 day) AND DATE_ADD(date_of_last_view, INTERVAL 29 day))
        AND (adobe_date BETWEEN report_start_date AND report_end_date)
        AND (num_views_started = 1)
        AND NOT(
            (COALESCE(stream_type,"NULL") = 'trailer') 
            AND (
                COALESCE(LOWER(vdo_initiate),"NULL") LIKE ('%auto%play%')
                --From 2023-03-01 onwards vdo_initiate=n/a will be considered as Auto play
                OR 
                (adobe_date >= '2023-03-01' AND COALESCE(LOWER(vdo_initiate),"NULL") = 'n/a')
            )
        )
    )
SELECT COUNT(DISTINCT Lapsing_Save_Num.adobe_tracking_id) / COUNT(DISTINCT Lapsing_Save_Denom.adobe_tracking_id)
FROM audience a 
LEFT JOIN Lapsing_Save_Denom
    ON a.adobe_tracking_id = Lapsing_Save_Denom.adobe_tracking_id
LEFT JOIN Lapsing_Save_Num
    ON Lapsing_Save_Denom.adobe_tracking_id = Lapsing_Save_Num.adobe_tracking_id
```

### Free-to-Paid Rate
- Percentage of users with a nonpaying status sometime in the reporting period that then possess a paying account status with an entitlement shift sometime in the reporting period
- Num: Users with an entitlement change flag and a paying status in the current month
- Denom: Users with a nonpaying status in the current month
- Dependencies: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`

```
WITH 
    Free_To_Paid_Denom AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` USER
        WHERE (paying_account_flag = 'NonPaying')
        AND (USER.report_date BETWEEN report_start_date AND report_end_date )
    )
    , Free_To_Paid_Num AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM
        (
            SELECT  report_date
                ,adobe_tracking_id
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` USER
            WHERE (entitlement_change_flag IN ('Upgrade: Free to Premium' , 'Upgrade: Free to Premium+'))
            AND (paying_account_flag = 'Paying')
            AND (USER.report_date BETWEEN report_start_date AND report_end_date)
        )
    )
SELECT COUNT(DISTINCT Free_To_Paid_Num.adobe_tracking_id) / COUNT(DISTINCT Free_To_Paid_Denom.adobe_tracking_id)
FROM audience a 
LEFT JOIN Free_To_Paid_Denom
    ON a.adobe_tracking_id = Free_To_Paid_Denom.adobe_tracking_id
LEFT JOIN Free_To_Paid_Num
    ON Free_To_Paid_Denom.adobe_tracking_id = Free_To_Paid_Num.adobe_tracking_id
```

### New Upgrade Rate
- Percentage of users who had never paid before who shifting to a paying status sometime in the reporting period
- Num: Users with their first change to paying status in the current month
- Denom: Users with no history of a paying status
- Dependencies: `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`

```
WITH 
    Net_New_Upgrade_Denom AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
        WHERE (first_paying_date IS NULL)
        AND (report_date BETWEEN report_start_date AND report_end_date) 
    )
    , Net_New_Upgrade_Num AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
        WHERE (paying_account_flag = 'Paying')
        AND (report_date BETWEEN report_start_date AND report_end_date)
    )
SELECT COUNT(DISTINCT Net_New_Upgrade_Num.adobe_tracking_id) / COUNT(DISTINCT Net_New_Upgrade_Denom.adobe_tracking_id)
FROM audience a 
LEFT JOIN Net_New_Upgrade_Denom
    ON a.adobe_tracking_id = Net_New_Upgrade_Denom.adobe_tracking_id
LEFT JOIN Net_New_Upgrade_Num
    ON Net_New_Upgrade_Denom.adobe_tracking_id = Net_New_Upgrade_Num.adobe_tracking_id
```

### Paid Winback Rate
- Percentage of users who had paid before but with a nonpaying status in the current month that have no shifted to a paying status sometime in the reporting month
- Num: Users with a paying status in the current month who had a nonpaying status in the currrent month and previously been paying sometime in their history
- Denom: Users with a nonpaying status in the current month and previously been paying sometime in their history
- Dependencies: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`, `upgrade_date_rank`
```
WITH 
    Paid_Winbacks_Denom AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM
        (
            SELECT  adobe_tracking_id
                ,report_date
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
            WHERE paying_account_flag = 'NonPaying'
            AND report_date BETWEEN report_start_date AND report_end_date 
        )
        WHERE adobe_tracking_id NOT IN ( 
            SELECT DISTINCT adobe_tracking_id 
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
            WHERE (first_paying_date IS NULL)
            AND (report_date BETWEEN report_start_date AND report_end_date)
        ) 
    )
    , Paid_Winbacks_Num AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE (paying_account_flag = 'Paying')
        AND (report_date BETWEEN report_start_date AND report_end_date) 
    )
SELECT COUNT(DISTINCT Paid_Winbacks_Num.adobe_tracking_id) / COUNT(DISTINCT Paid_Winbacks_Denom.adobe_tracking_id)
FROM audience a 
LEFT JOIN Paid_Winbacks_Denom
    ON a.adobe_tracking_id = Paid_Winbacks_Denom.adobe_tracking_id
LEFT JOIN Paid_Winbacks_Num
    ON Paid_Winbacks_Denom.adobe_tracking_id = Paid_Winbacks_Num.adobe_tracking_id
```

### Cancellation Save Rate (prev. Paid Churn Save Rate)
- Percentage of users who turned off auto renew sometime in the reporting period who then turn it back on
- Num: Users with a paying account and auto renew turned on who had previously had a paying account and auto renew off in the current month
- Denom: Users with a paying account and auto renew off in the current month
- Dependencies: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`

```
WITH 
    Cancel_Save_Denom AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE (paying_account_flag = 'Paying')
        AND (auto_renew_flag = 'OFF')
        AND (report_date BETWEEN report_start_date AND report_end_date )
    )
    , Cancel_Save_Num AS (
        SELECT  DISTINCT adobe_tracking_id
        FROM
        (
            SELECT  adobe_tracking_id
                ,report_date
                ,auto_renew_flag                                                                      AS auto_renew_flag_today
                ,LEAD(auto_renew_flag,1) OVER ( PARTITION BY adobe_tracking_id ORDER BY report_date ) AS auto_renew_flag_next_day
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
            WHERE report_date BETWEEN report_start_date AND report_end_date
            ORDER BY 1, 2 
        )
        WHERE (auto_renew_flag_today = 'OFF')
        AND (auto_renew_flag_next_day = 'ON')
    )
SELECT COUNT(DISTINCT Cancel_Save_Num.adobe_tracking_id) / COUNT(DISTINCT Cancel_Save_Denom.adobe_tracking_id)
FROM audience a 
LEFT JOIN Cancel_Save_Denom
    ON a.adobe_tracking_id = Cancel_Save_Denom.adobe_tracking_id
LEFT JOIN Cancel_Save_Num
    ON Cancel_Save_Denom.adobe_tracking_id = Cancel_Save_Num.adobe_tracking_id
```

### Paid Churn Rate
- Percentage of users who have churned sometime in the reporting period who had been paying 
- Num: Users with a paid flag and a churn flag at the end of the month
- Denom: User with a paid flag at the end of the month
- Dependencies: `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`

```
WITH 
    EOM_Paid_Churn_Denom AS  (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
        WHERE (base_date = report_end_date)
        AND (entitlement = 'Paid')
    )
    , EOM_Paid_Churn_Num AS  (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
        WHERE (base_date = report_end_date)
        AND (entitlement = 'Paid')
        AND (Churn_flag = 'Churn')
    )
SELECT COUNT(DISTINCT EOM_Paid_Churn_Num.adobe_tracking_id) / COUNT(DISTINCT EOM_Paid_Churn_Denom.adobe_tracking_id)
FROM audience a 
LEFT JOIN EOM_Paid_Churn_Denom
    ON a.adobe_tracking_id = EOM_Paid_Churn_Denom.adobe_tracking_id
LEFT JOIN EOM_Paid_Churn_Num
    ON EOM_Paid_Churn_Denom.adobe_tracking_id = EOM_Paid_Churn_Num.adobe_tracking_id
```

# Changelog

### V2 to V3

- Code Changes (no effect on results)
    - Engager defintion split into subquery
    - General code refactoring
- Measurement Changes
    - Transactional filter expanded to include all transactional emails beyond October Privacy Transcom Emails
    - Email Engagers definition change. 
        - Before: Any user with an user email open (excludes machine opens) in their user history before the end of the reporting period, excluding transactional emails
        - After: Any user with email delivered or joined the platform in a 4 month period before the start of the relevant holdout period to the end of the reporting period.
            - For example, a February report contains users who had an email delivery or registration date between September 1st (4 months before Jan 1st, the start of relevant holdout period) to February 28th (end of reporting period).
        - Justification: Email deliveries may leave impression on users, leaving the intended effect of marketing. New users are sent emails, so they are included in measurement. 4 month period chosen to minimize impact of communication blackout due to previous holdout.
    - Viewing winback definition restricted to 30 to 90 day window in both numerator and denominator. Previous version numerator had no restriction
    - Paid churn definition changed from end of previous reporting month's cohort to end of current reporting month's cohort

### V3 to V4

- Code changes (no effect on results)
    - Production Prep: Unsub logic changed to enable backfilling
    - Code cleanup: Email targeted defintion restricted to exclude transactional emails in reporting period
    - Code cleanup: Holdout definition changed to exclude any email targeting instead of repeating subquery
    - Code cleanup: Engager and new user split out
    - Code cleanup: Ensure adobe_tracking_id is distinct in base table by pulling distinct ids in Save_Denom and Winback_Denom into separate tables before joining
    - Syntax update: "Paid Churn Save Rate" changed to "Cancellation Save Rate"
    - Code cleanup: Changed KPI numerator and denominators to binary
    - Other general refactoring
- Measurement Changes
    - BUGFIX: Holdout group email received exclusion logic fixed to correctly exclude users from holdout group if email was received
    - Upgrade definition changed to exclude Premium to Premium+ upgrades. Standard upgrade definition restricted from Nonpaying to Paying, not an upgrade in paying status